## Request NOAA Daily Meterology Data (BATCH).

*** FOR SINGLE SITE REQUESTS USE acis_data_request.ipynb ***

This notebook is an example of how to use pydrology to download and resample NOAA SC-ACIS. NOAA provides daily meteorology data, which can be accessed on their website (https://scacis.rcc-acis.org/). The data requests in this notebook come from Single Station Products > Daily Data Listing. The data comes from a network of weather monitoring stations that may have different meterology options, most often temperature and precipitation. Weather stations may have different time periods of data due to when they were put online or taken off. If your request contains a longer time period than is available, no-data values will be used as placeholders for the missing data.

The general workflow for requesting USGS streamflow data is as follows:
1. Request the MET data for a particular site using the function call below.
2. Inspect the raw data for missing values and other issues.
3. Handle any data cleaning and then the data is ready for use!
4. (Optional) Downsample or upsample the data using the provided functions.

In [ ]:
# Library imports.
import numpy as np
import matplotlib.pyplot as plt

# Local imports.
from pydrology.acis.acis_request import request_acis_data
from pydrology import time_series

## 1) Request meteorology data.

In [ ]:
# Site IDs to request data for.
# Add <space>6 to the end of GHCN site IDs. Add <space>2 for coop site ids. (E.g., "USS0020A23S" => "USS0020A23S 6")
site_ids = ['US1NYTM0047 6', 'USC00304996 6', 'US1NYSF0145 6', 'US1NYTM0003 6', 'US1NYMG0003 6', 'US1NYSF0074 6', 'US1NYUL0015 6', 'USC00306062 6', 'US1NYER0234 6', 'USW00014748 6', 'US1NYCB0015 6', 'US1NYTG0008 6', 'US1NYLV0001 6', 'US1NYTM0005 6', 'US1NYTM0038 6', 'US1NYHR0023 6', 'USC00304952 6', 'US1NYBM0021 6', 'USC00302197 6', 'US1NYCQ0041 6', 'US1NYST0044 6', 'US1NYCY0034 6', 'US1NYHM0011 6', 'US1NYSR0040 6', 'US1NYHM0009 6', 'US1NYCY0005 6', 'US1NYER0178 6', 'USC00305248 6', 'US1NYDT0032 6', 'US1NYSR0015 6', 'US1NYER0245 6', 'USC00300707 6', 'US1NYNS0027 6', 'US1NYGR0007 6', 'US1NYHR0020 6', 'USC00302036 6', 'US1NYSF0103 6', 'US1NYOG0028 6', 'US1NYOS0015 6', 'US1NYOD0060 6', 'USC00308412 6', 'USC00308962 6', 'US1NYER0211 6', 'USC00304102 6', 'US1NYCM0010 6', 'USC00306196 6', 'US1NYOG0052 6', 'US1NYGR0014 6', 'US1NYRN0001 6', 'US1NYRC0002 6', 'US1NYSR0032 6', 'US1NYST0003 6', 'US1NYMR0015 6', 'US1NYNS0007 6', 'USC00309389 6', 'US1NYER0053 6', 'USC00306314 6', 'US1NYDT0029 6', 'US1NYMR0055 6', 'US1NYWR0010 6', 'US1NYSR0029 6', 'USC00306480 6', 'US1NYCN0016 6', 'USC00305715 6', 'US1NYCL0007 6', 'USC00308600 6', 'US1NYWR0021 6', 'US1NYRN0015 6', 'US1NYCY0002 6', 'USR0000NGAN 6', 'USW00094790 6', 'US1NYOG0070 6', 'US1NYDT0035 6', 'US1NYAB0056 6', 'USC00303955 6', 'US1NYAB0021 6', 'USC00300870 6', 'US1NYUL0029 6', 'US1NYSR0057 6', 'US1NYDL0025 6', 'USW00014771 6', 'US1NYTG0028 6', 'USC00303983 6', 'US1NYDT0023 6', 'USW00094789 6', 'US1NYNG0055 6', 'US1NYOR0002 6', 'USC00305679 6', 'USC00309292 6', 'USW00004725 6', 'USC00303851 6', 'US1NYCM0017 6', 'US1NYER0063 6', 'US1NYSR0016 6', 'US1NYDT0012 6', 'USC00300317 6', 'US1NYNS0065 6', 'US1NYAB0032 6', 'US1NYSF0002 6', 'USW00054780 6', 'US1NYSR0022 6', 'US1NYHM0001 6', 'US1NYER0150 6', 'USC00305597 6', 'USW00004789 6', 'USW00004781 6', 'US1NYER0166 6', 'US1NYRN0023 6', 'USC00308578 6', 'US1NYAB0046 6', 'US1NYES0005 6', 'US1NYER0158 6', 'USC00308961 6', 'US1NYCM0024 6', 'US1NYES0006 6', 'USW00004724 6', 'USC00304912 6', 'US1NYHM0010 6', 'US1NYDL0034 6', 'US1NYCL0012 6', 'USC00304684 6', 'USC00300055 6', 'US1NYJF0026 6', 'US1NYOT0011 6', 'US1NYOL0005 6', 'USW00014747 6', 'US1NYAB0066 6', 'USC00303773 6', 'US1NYWS0010 6', 'USW00064776 6', 'USC00302610 6', 'USC00304537 6', 'USC00304575 6', 'US1NYER0242 6', 'USC00305840 6', 'US1NYLW0003 6', 'US1NYOD0002 6', 'US1NYST0045 6', 'USW00014768 6', 'USC00308932 6', 'US1NYER0151 6', 'US1NYOT0030 6', 'USW00014719 6', 'US1NYMG0001 6', 'US1NYST0033 6', 'USC00307705 6', 'US1NYER0059 6', 'US1NYOD0004 6', 'US1NYER0208 6', 'US1NYSF0131 6', 'US1NYWN0023 6', 'USC00301625 6', 'US1NYLW0002 6', 'US1NYCY0014 6', 'US1NYSF0085 6', 'US1NYOG0061 6', 'US1NYCB0009 6', 'USW00094745 6', 'USC00309593 6', 'US1NYCM0021 6', 'US1NYCQ0039 6', 'USC00300910 6', 'USC00301580 6', 'USC00307274 6', 'US1NYCB0024 6', 'USC00301068 6', 'US1NYNG0047 6', 'USC00305280 6', 'US1NYCN0002 6', 'US1NYWC0034 6', 'US1NYLV0005 6', 'USC00300331 6', 'USC00307134 6', 'US1NYFL0009 6', 'USC00306774 6', 'USC00304207 6', 'US1NYCY0026 6', 'US1NYUL0039 6', 'US1NYCB0023 6', 'USW00094704 6', 'USW00014733 6', 'US1NYER0098 6', 'US1NYSF0129 6', 'USC00306047 6', 'USC00303961 6', 'US1NYTG0015 6', 'USC00305236 6', 'US1NYMR0065 6', 'US1NYBM0052 6', 'US1NYNG0032 6', 'US1NYSF0092 6', 'US1NYDL0032 6', 'US1NYRN0020 6', 'US1NYER0189 6', 'US1NYOS0001 6', 'USC00309072 6', 'US1NYOT0031 6', 'US1NYSL0017 6', 'US1NYTG0030 6', 'US1NYSF0084 6', 'US1NYCQ0042 6', 'US1NYWC0011 6', 'US1NYWC0022 6', 'US1NYSR0004 6', 'US1NYER0194 6', 'USC00308910 6', 'USC00306376 6', 'USC00300608 6', 'USC00307750 6', 'US1NYQN0039 6', 'USW00054778 6', 'USC00306465 6', 'USC00307329 6', 'US1NYER0039 6', 'US1NYSR0059 6', 'US1NYFL0007 6', 'US1NYBM0024 6', 'USC00307520 6', 'US1NYES0011 6', 'US1NYCQ0006 6', 'USC00308577 6', 'US1NYHM0008 6', 'USC00309442 6', 'USC00300183 6', 'US1NYWR0017 6', 'US1NYER0050 6', 'US1NYMR0026 6', 'US1NYER0054 6', 'USC00303088 6', 'US1NYBM0014 6', 'USC00300379 6', 'US1NYFK0007 6', 'US1NYOT0026 6', 'US1NYGN0023 6', 'US1NYSF0073 6', 'US1NYYT0012 6', 'USW00014735 6', 'USC00300093 6', 'USC00300668 6', 'US1NYUL0019 6', 'USC00300500 6', 'US1NYRN0013 6', 'US1NYSF0044 6', 'US1NYSF0070 6', 'US1NYSF0089 6', 'USC00304808 6', 'US1NYER0236 6', 'US1NYSF0079 6', 'US1NYSY0011 6', 'US1NYWC0037 6', 'US1NYMG0005 6', 'US1NYCB0016 6', 'US1NYMR0078 6', 'US1NYMR0084 6', 'USC00306164 6', 'US1NYER0086 6', 'US1NYST0041 6', 'US1NYBM0056 6', 'US1NYGN0020 6', 'USC00300684 6', 'US1NYSF0007 6', 'USC00309047 6', 'USC00303444 6', 'US1NYCM0006 6', 'USW00094740 6', 'US1NYSF0016 6', 'US1NYST0010 6', 'US1NYER0102 6', 'US1NYMG0002 6', 'US1NYGR0006 6', 'USC00306525 6', 'US1NYWN0006 6', 'US1NYDL0028 6', 'US1NYSF0080 6', 'US1NYOR0017 6', 'USC00300443 6', 'US1NYWS0008 6', 'USC00307780 6', 'US1NYMD0022 6', 'US1NYSC0032 6', 'USC00301974 6', 'USC00303184 6', 'US1NYSF0114 6', 'US1NYSL0006 6', 'USC00308132 6', 'US1NYSF0134 6', 'US1NYDT0008 6', 'US1NYNS0042 6', 'US1NYOG0071 6', 'US1NYSF0034 6', 'US1NYAB0001 6', 'US1NYHR0018 6', 'USC00305606 6', 'USC00300220 6', 'USR0000NSHR 6', 'US1NYCT0025 6', 'USC00307484 6', 'USW00014757 6', 'US1NYCT0030 6', 'US1NYWY0007 6', 'USC00300343 6', 'US1NYHM0006 6', 'USC00303087 6', 'US1NYSC0017 6', 'US1NYCQ0022 6', 'USC00305512 6', 'US1NYNG0012 6', 'US1NYCY0008 6', 'USC00304565 6', 'USC00309425 6', 'USW00054746 6', 'US1NYAB0010 6', 'USC00301708 6', 'US1NYTG0005 6', 'US1NYNG0044 6', 'USC00307425 6', 'US1NYWC0019 6', 'US1NYPT0002 6', 'US1NYWC0006 6', 'US1NYRL0008 6', 'US1NYTM0027 6', 'US1NYUL0037 6', 'US1NYGN0013 6', 'US1NYCB0019 6', 'US1NYAB0023 6', 'US1NYLW0013 6', 'USC00309248 6', 'US1NYQN0033 6', 'US1NYSC0027 6', 'USC00301265 6', 'US1NYTM0042 6', 'USC00303322 6', 'US1NYDL0023 6', 'USW00054757 6', 'USC00301211 6', 'US1NYER0072 6', 'USR0000NSTO 6', 'USW00094728 6', 'USW00014732 6', 'US1NYRC0001 6', 'US1NYJF0030 6', 'USC00305233 6', 'USC00308248 6', 'US1NYES0012 6', 'US1NYWC0018 6', 'US1NYUL0031 6', 'USC00306747 6', 'USR0000NSCH 6', 'US1NYMD0010 6', 'US1NYSC0031 6', 'USC00308721 6', 'USC00301309 6', 'USC00302934 6', 'US1NYUL0028 6', 'US1NYDT0034 6', 'USC00309049 6', 'USW00094725 6', 'US1NYSF0123 6', 'US1NYBM0007 6', 'US1NYSF0153 6', 'USC00304844 6', 'US1NYNG0027 6', 'US1NYER0135 6', 'USC00300364 6', 'USW00054790 6', 'USC00309499 6', 'US1NYER0075 6', 'US1NYOG0012 6', 'USC00301388 6', 'US1NYSF0062 6', 'US1NYON0017 6', 'US1NYST0030 6', 'US1NYOG0046 6', 'US1NYWC0035 6', 'US1NYMR0023 6', 'US1NYTG0026 6', 'US1NYOG0010 6', 'USC00300063 6', 'US1NYUL0020 6', 'US1NYAL0011 6', 'US1NYOR0004 6', 'US1NYER0096 6', 'USC00304555 6', 'USC00301787 6', 'US1NYBM0004 6', 'US1NYLW0009 6', 'USC00300785 6', 'US1NYSF0060 6', 'US1NYJF0044 6', 'US1NYER0051 6', 'US1NYAB0047 6', 'USC00309089 6', 'US1NYKN0025 6', 'USC00303346 6', 'US1NYTM0023 6', 'US1NYON0022 6', 'US1NYMR0050 6', 'US1NYER0057 6', 'US1NYQN0026 6', 'US1NYOD0066 6', 'USW00054773 6', 'US1NYGR0015 6', 'US1NYER0205 6', 'US1NYER0203 6', 'US1NYOR0021 6', 'US1NYWN0026 6', 'US1NYWN0018 6', 'US1NYOD0061 6', 'US1NYNG0002 6', 'US1NYUL0034 6', 'US1NYCQ0035 6', 'US1NYNS0034 6', 'US1NYSC0026 6', 'US1NYTM0045 6', 'USC00300937 6', 'US1NYCM0013 6', 'US1NYTM0018 6', 'US1NYNS0066 6', 'US1NYAL0002 6', 'USC00308839 6', 'US1NYSF0127 6', 'US1NYHR0016 6', 'USC00307842 6', 'US1NYOT0035 6', 'US1NYDT0037 6', 'US1NYOD0067 6', 'USC00301102 6', 'US1NYSF0138 6', 'US1NYUL0023 6', 'US1NYOD0043 6', 'US1NYAB0041 6', 'US1NYSC0016 6', 'US1NYYT0008 6', 'US1NYSR0046 6', 'US1NYRC0016 6', 'US1NYOR0018 6', 'USC00300085 6', 'USC00300889 6', 'US1NYON0018 6', 'US1NYCQ0005 6', 'USC00302454 6', 'US1NYSY0001 6', 'US1NYCT0022 6', 'US1NYWR0015 6', 'US1NYMR0021 6', 'USW00054787 6', 'US1NYWC0032 6', 'US1NYCR0003 6', 'USW00004720 6', 'US1NYOR0019 6', 'US1NYER0219 6', 'USC00305925 6', 'USW00064775 6', 'US1NYOD0019 6', 'US1NYGN0006 6', 'USC00306660 6', 'USC00301152 6', 'US1NYNS0046 6', 'US1NYSF0077 6', 'US1NYBM0055 6', 'USC00304174 6', 'US1NYNG0030 6', 'US1NYUL0038 6', 'US1NYOD0023 6', 'US1NYSF0148 6', 'USC00307749 6', 'US1NYDT0024 6', 'USC00303050 6', 'USC00307742 6', 'US1NYCR0001 6', 'US1NYDT0010 6', 'US1NYSC0030 6', 'US1NYCY0035 6', 'US1NYCT0002 6', 'USC00304564 6', 'US1NYSC0002 6', 'USC00300321 6', 'US1NYER0177 6', 'USW00014750 6', 'US1NYER0138 6', 'US1NYMD0016 6', 'USC00301752 6', 'USC00306276 6', 'US1NYER0066 6', 'US1NYSF0049 6', 'US1NYYT0013 6', 'USC00301264 6', 'US1NYSF0110 6', 'US1NYTM0004 6', 'US1NYUL0016 6', 'USC00305714 6', 'US1NYBM0051 6', 'US1NYER0104 6', 'US1NYER0122 6', 'USC00306085 6', 'US1NYSF0100 6']



# list of met elements to request.
# Options in dictionary below:
met_elements_ref = {'maxt': 'MaxTemp', 'mint': 'MinTemp', 'avgt': 'AvgTemp', 'pcpn': 'Precipitation',
                 'snow': 'SnowFall', 'snwd': 'SnowDepth'}
met_elements = ['maxt', 'mint', 'snow', 'avgt', 'pcpn', 'snwd']
met_cols = [met_elements_ref[i] for i in met_elements]

# Date range.
start_date = '1970-01-01'  # YYYY-mm-dd.
end_date = '2023-01-09'

## Make the request.
acis_df_list = []
for i, site_id in enumerate(site_ids):
    print(f'>>>Processing {i} out of {len(site_ids)}')
    try:
        acis_df = request_acis_data(met_elements, site_id, start_date, end_date)

        # Don't include data frames if they don't have enough data over the time window.
        met_col_df = acis_df[met_cols].copy()
        met_col_df.replace('M', np.nan, inplace=True)
        all_na_idx = met_col_df.index[met_col_df.isnull().all(1)]
        if len(all_na_idx) / acis_df.shape[0] < 0.05: # 5% threshold.
            print('{:.4g}% of data are NA ***'.format(len(all_na_idx) / acis_df.shape[0] * 100))
            acis_df_list.append(acis_df)
        else:
            print('{:.4g}% of data are NA'.format(len(all_na_idx) / acis_df.shape[0] * 100))
    except Exception as e:
        print(site_id)
        print(e)

    # if i == 100:
    #     break

In [ ]:
print(len(acis_df_list))

## 2) Data cleaning (Optional).

In [ ]:
dt = 1440 # Time step in minutes.
data_columns = ['MaxTemp', 'MinTemp', 'SnowFall', 'AvgTemp', 'Precipitation', 'SnowFall', 'SnowDepth']
time_column = 'Date'

### Set Trace to Zero. 'T' => 0.0

In [ ]:
# Trace means that there was precipitation, but it was below the measurement threshold.
trace_value = 'T'
for acis_df in acis_df_list:
    acis_df.replace(trace_value, 0.0, inplace=True)

### Set Missing to NaN. 'M' => NaN

In [ ]:
missing_value = 'M'
# Replace M with NaN and remove rows where all variables of interest are 'M'
for acis_df in acis_df_list:
    acis_df.replace(missing_value, np.nan, inplace=True)

### Add NaN values to missing time steps.

In [ ]:
for acis_df in acis_df_list:
    acis_df = time_series.standardize_datetime(acis_df, time_column, data_columns, dt)

### Interpolation of NaN values.

In [ ]:
for acis_df in acis_df_list:
    acis_df = time_series.interpolate_time_series(acis_df, data_columns, method='linear')

## 4) Data resampling (Optional).

In [ ]:
new_dt = 360 # New time step in minutes.
resample_acis_df = time_series.resample_data(acis_df, time_column, data_columns, new_dt)
resample_acis_df.head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(acis_df[time_column], acis_df['MaxTemp'], 'b-o', label='Orig. Data')
ax.plot(resample_acis_df[time_column], resample_acis_df['MaxTemp'], 'rx', label='Clean/Resamp.')
ax.legend()
ax.tick_params(axis='x', rotation=45)
plt.show()

## 5) Save Data Frame.

In [ ]:
for i, acis_df in enumerate(acis_df_list):
    acis_df.to_csv(f'~/Path to Directory/{site_ids[i]}_{start_date}_{end_date}_Daily_Met.csv', index=False)

